<a href="https://colab.research.google.com/github/fanyak/portfolio/blob/main/backpack_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SOURCE CODE: https://www.kaggle.com/code/faniakritidoy/feature-engineering-with-rapids-lb-38-847/edit

# FEATURE ENGINEERING

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
!head -n 10 train.csv

id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611722805222309,112.15875
1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.07853658053123,68.88056
2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643759949103497,39.1732
3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220306632067,80.60793
4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338465908988,86.02312
5,Nike,Canvas,Medium,10.0,No,Yes,,Black,7.241812431393921,20.01553
6,Nike,,Large,3.0,No,No,Backpack,Green,6.82812289959413,84.805
7,Puma,Canvas,Small,1.0,Yes,Yes,Backpack,Blue,21.48886449943962,27.15815
8,Under Armour,Polyester,Medium,8.0,Yes,No,Tote,Gray,10.207780204196547,25.98652


In [ ]:
train_df = pd.read_csv('train.csv', index_col='id', header =0)
train_df.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


In [ ]:
test_df = pd.read_csv('test.csv', index_col='id', header =0)
test_df.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
id,,,,,,,,,
300000,Puma,Leather,Small,2.0,No,No,Tote,Green,20.671147
300001,Nike,Canvas,Medium,7.0,No,Yes,Backpack,Green,13.564105
300002,Adidas,Canvas,Large,9.0,No,Yes,Messenger,Blue,11.809799
300003,Adidas,Nylon,Large,1.0,Yes,No,Messenger,Green,18.477036
300004,NaN,Nylon,Large,2.0,Yes,Yes,Tote,Black,9.907953


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300000 entries, 0 to 299999
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Brand                 290295 non-null  object 
 1   Material              291653 non-null  object 
 2   Size                  293405 non-null  object 
 3   Compartments          300000 non-null  float64
 4   Laptop Compartment    292556 non-null  object 
 5   Waterproof            292950 non-null  object 
 6   Style                 292030 non-null  object 
 7   Color                 290050 non-null  object 
 8   Weight Capacity (kg)  299862 non-null  float64
 9   Price                 300000 non-null  float64
dtypes: float64(3), object(7)
memory usage: 25.2+ MB


In [ ]:
train_df.isna().sum()

,0
Brand,9705
Material,8347
Size,6595
Compartments,0
Laptop Compartment,7444
Waterproof,7050
Style,7970
Color,9950
Weight Capacity (kg),138
Price,0


In [ ]:
test_df.isna().sum()

,0
Brand,6227
Material,5613
Size,4381
Compartments,0
Laptop Compartment,4962
Waterproof,4811
Style,5153
Color,6785
Weight Capacity (kg),77


In [ ]:
train_df.select_dtypes(include='object')

,Brand,Material,Size,Laptop Compartment,Waterproof,Style,Color
id,,,,,,,
0,Jansport,Leather,Medium,Yes,No,Tote,Black
1,Jansport,Canvas,Small,Yes,Yes,Messenger,Green
2,Under Armour,Leather,Small,Yes,No,Messenger,Red
3,Nike,Nylon,Small,Yes,No,Messenger,Green
4,Adidas,Canvas,Medium,Yes,Yes,Messenger,Green
...,...,...,...,...,...,...,...
299995,Adidas,Leather,Small,No,No,Tote,Blue
299996,Jansport,Leather,Large,No,Yes,Tote,Blue
299997,Puma,Canvas,Large,Yes,Yes,Backpack,Pink


In [ ]:
def clean(X):
  df = X.copy()
  df.columns = df.columns.str.lower().str.replace(' ', '_')
  return df

In [ ]:
# def yes_no_encoding(X):
#   df = X.copy()
#   for col in X.select_dtypes(include='object'):
#     values = np.strings.lower(df[col].dropna().unique().astype(str))
#     if pd.Series(values).isin(['yes', 'no']).all():
#       #df.loc[:, col] = df[col].astype("category").cat.set_categories(['No', 'Yes']).cat.codes.map(lambda x: np.nan if x == -1 else int(x))
#       df.loc[:, col]=df[col].map({"Yes":1, "No":0, np.nan: np.nan})
#   return df

In [ ]:
# # encode ordinals
# def label_encoding(X, col):
#   df = X.copy()
#   return df

In [ ]:
# def load_data(target):
#   train_df = pd.read_csv('train.csv', index_col='id', header=0)
#   test_df = pd.read_csv('test.csv', index_col='id', header=0)
#   y = train_df[[target]]
#   df =  pd.concat([train_df.drop(columns=[target]), test_df], axis=0)
#   df = clean(df)
#   y = clean(y)
#   #df = yes_no_encoding(df)
#   train = df.iloc[train_df.index, :]
#   test = df.iloc[test_df.index, :]
#   return train.join(y), test

In [ ]:
#train, test = load_data('Price')

In [ ]:
#train_df.index, train.index
test_df.index, test.index

(Index([300000, 300001, 300002, 300003, 300004, 300005, 300006, 300007, 300008,
        300009,
        ...
        499990, 499991, 499992, 499993, 499994, 499995, 499996, 499997, 499998,
        499999],
       dtype='int64', name='id', length=200000),
 Index([300000, 300001, 300002, 300003, 300004, 300005, 300006, 300007, 300008,
        300009,
        ...
        404252, 404253, 404254, 404255, 404256, 404257, 404258, 404259, 404260,
          4042],
       dtype='int64', name='id', length=104262))

# FEATURE ENGINEERING

In [ ]:
def load_data():
  train_df = pd.read_csv('train.csv', index_col='id', header=0)
  test_df = pd.read_csv('test.csv', index_col='id', header=0)
  return train_df.copy(), test_df.copy()


## INTERACTIONS WITH CATEGORICALS

In [ ]:
CATS = train_df.select_dtypes(include='object').columns.tolist()
CATS

['Brand',
 'Material',
 'Size',
 'Laptop Compartment',
 'Waterproof',
 'Style',
 'Color']

In [ ]:
#Interaction with categorical
train_alt, test_alt = load_data()
COMBO = []
for i,c in enumerate(list(train_alt.columns[0:-2])):
    print(f"{c}, ", end="")
    combine = pd.concat([train_alt[c], test_alt[c]], axis = 0)
    # 1. first convert categorical to numericcal
    # SOS: this will transform n/a to -1!!!!!!!!!!!!
    combine,_ = pd.factorize(combine)
    train_alt[c] = combine[:len(train_alt)]
    test_alt[c] = combine[len(train_alt):]
    n = f"{c}_wc"
    # 2. create the interaction
    train_alt[n] = train_alt[c]*100 + train_alt["Weight Capacity (kg)"]
    test_alt[n] = test_alt[c]*100 + test_alt["Weight Capacity (kg)"]
    COMBO.append(n)
print()
print(f"We engineer {len(COMBO)} new columns!")
print( COMBO )

Brand, Material, Size, Compartments, Laptop Compartment, Waterproof, Style, Color, 
We engineer 8 new columns!
['Brand_wc', 'Material_wc', 'Size_wc', 'Compartments_wc', 'Laptop Compartment_wc', 'Waterproof_wc', 'Style_wc', 'Color_wc']


In [ ]:
train_alt.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price,Brand_wc,Material_wc,Size_wc,Compartments_wc,Laptop Compartment_wc,Waterproof_wc,Style_wc,Color_wc
id,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,11.611723,112.15875,11.611723,11.611723,11.611723,11.611723,11.611723,11.611723,11.611723,11.611723
1,0,1,1,1,0,1,1,1,27.078537,68.88056,27.078537,127.078537,127.078537,127.078537,27.078537,127.078537,127.078537,127.078537
2,1,0,1,2,0,0,1,2,16.643760,39.17320,116.643760,16.643760,116.643760,216.643760,16.643760,16.643760,116.643760,216.643760
3,2,2,1,3,0,0,1,1,12.937220,80.60793,212.937220,212.937220,112.937220,312.937220,12.937220,12.937220,112.937220,112.937220
4,3,1,0,4,0,1,1,1,17.749338,86.02312,317.749338,117.749338,17.749338,417.749338,17.749338,117.749338,117.749338,117.749338


In [ ]:
train_alt.dtypes

,0
Brand,int64
Material,int64
Size,int64
Compartments,int64
Laptop Compartment,int64
Waterproof,int64
Style,int64
Color,int64
Weight Capacity (kg),float64
Price,float64


In [ ]:
train_alt.isna().sum()

,0
Brand,0
Material,0
Size,0
Compartments,0
Laptop Compartment,0
Waterproof,0
Style,0
Color,0
Weight Capacity (kg),138
Price,0


In [ ]:
train_alt.describe()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price,Brand_wc,Material_wc,Size_wc,Compartments_wc,Laptop Compartment_wc,Waterproof_wc,Style_wc,Color_wc
count,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,299862.000000,300000.000000,299862.000000,299862.000000,299862.000000,299862.000000,299862.000000,299862.000000,299862.000000,299862.000000
mean,1.908153,1.465843,0.945157,4.487933,0.455900,0.470090,0.937277,2.449117,18.029994,81.411107,208.875783,164.639768,112.564480,466.843448,63.637974,65.066631,111.779119,262.955326
std,1.479240,1.198368,0.868000,2.871608,0.545603,0.544157,0.861587,1.803114,6.966914,39.039340,148.068337,120.022023,87.041977,287.251385,54.947167,54.832892,86.417842,180.453716
min,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,5.000000,15.000000,-95.000000,-95.000000,-95.000000,5.000000,-95.000000,-95.000000,-95.000000,-95.000000
25%,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,12.097867,47.384620,109.868443,27.802841,22.150977,217.918667,17.078906,17.416359,22.716358,115.835197
50%,2.000000,2.000000,1.000000,4.000000,0.000000,0.000000,1.000000,3.000000,18.068614,80.956120,216.284810,205.000000,116.878354,429.025735,28.908154,29.744414,116.883780,305.000000
75%,3.000000,3.000000,2.000000,7.000000,1.000000,1.000000,2.000000,4.000000,24.002375,115.018160,322.908086,306.997037,211.908844,717.360494,117.744367,117.947766,211.171213,418.988774
max,4.000000,3.000000,2.000000,9.000000,1.000000,1.000000,2.000000,5.000000,30.000000,150.000000,430.000000,330.000000,230.000000,930.000000,130.000000,130.000000,230.000000,530.000000


## TARGET ENCODING WITH MULTIPLE STATS

In [ ]:
from sklearn.model_selection import KFold

### Try it!

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
sanity_check = []
for k, (t_index, v_index) in enumerate(kf.split(train_alt)):
  print(f"Fold {k}:")
  print(f"  Train: index={t_index}")
  print(f"  Valid: index={v_index}")
  sanity_check.extend((list(v_index)))

Fold 0:
  Train: index=[     1      2      3 ... 299997 299998 299999]
  Valid: index=[     0      6     11 ... 299973 299983 299992]
Fold 1:
  Train: index=[     0      1      2 ... 299995 299996 299998]
  Valid: index=[     4     10     31 ... 299982 299997 299999]
Fold 2:
  Train: index=[     0      2      3 ... 299997 299998 299999]
  Valid: index=[     1      9     20 ... 299987 299988 299991]
Fold 3:
  Train: index=[     0      1      2 ... 299995 299997 299999]
  Valid: index=[     3      7      8 ... 299994 299996 299998]
Fold 4:
  Train: index=[     0      1      3 ... 299997 299998 299999]
  Valid: index=[     2      5     13 ... 299990 299993 299995]


In [ ]:
sanity_check.sort()
len(sanity_check), len(set(sanity_check)), train_alt.shape

(300000, 300000, (300000, 18))

In [ ]:
STATS = ["mean","std","count","nunique","median","min","max","skew"]
STATS2 = ["mean","std"]
tmp = train_alt.groupby("Weight Capacity (kg)")["Price"].agg(STATS)
tmp

,mean,std,count,nunique,median,min,max,skew
Weight Capacity (kg),,,,,,,,
5.000000,78.304714,39.384232,4286,4029,76.111625,15.00000,150.00000,0.110774
5.004444,132.156440,NaN,1,1,132.156440,132.15644,132.15644,NaN
5.008382,143.890560,NaN,1,1,143.890560,143.89056,143.89056,NaN
5.009731,144.075910,NaN,1,1,144.075910,144.07591,144.07591,NaN
5.010086,104.852040,0.738389,2,2,104.852040,104.32992,105.37416,NaN
...,...,...,...,...,...,...,...,...
29.996888,117.151670,NaN,1,1,117.151670,117.15167,117.15167,NaN
29.997986,68.255628,54.810005,4,4,52.133435,21.85457,146.90107,1.500809
29.998925,104.765930,NaN,1,1,104.765930,104.76593,104.76593,NaN


In [ ]:
train_alt[["Brand","Weight Capacity (kg)"]].merge(tmp, on="Weight Capacity (kg)")

,Brand,Weight Capacity (kg),mean,std,count,nunique,median,min,max,skew
0,0,11.611723,72.092237,31.405870,8,8,71.329080,23.81545,112.15875,-0.185196
1,0,27.078537,68.880560,NaN,1,1,68.880560,68.88056,68.88056,NaN
2,1,16.643760,39.173200,NaN,1,1,39.173200,39.17320,39.17320,NaN
3,2,12.937220,80.607930,NaN,1,1,80.607930,80.60793,80.60793,NaN
4,3,17.749338,86.023120,NaN,1,1,86.023120,86.02312,86.02312,NaN
...,...,...,...,...,...,...,...,...,...,...
299857,3,12.730812,84.687215,64.078405,2,2,84.687215,39.37694,129.99749,NaN
299858,0,26.633182,19.858190,NaN,1,1,19.858190,19.85819,19.85819,NaN
299859,4,11.898250,78.902730,40.259095,123,119,76.478350,17.15632,150.00000,0.167557
299860,3,6.175738,115.890800,NaN,1,1,115.890800,115.89080,115.89080,NaN


In [ ]:
train_alt[CATS].dtypes, train_alt[CATS].astype("category").dtypes # we need to tell the model that these are categories !!!!!!!!!!!

(Brand                 int64
 Material              int64
 Size                  int64
 Laptop Compartment    int64
 Waterproof            int64
 Style                 int64
 Color                 int64
 dtype: object,
 Brand                 category
 Material              category
 Size                  category
 Laptop Compartment    category
 Waterproof            category
 Style                 category
 Color                 category
 dtype: object)

### Implement It with KFold to prevent leakage!!!

In [ ]:
%%time

FOLDS = 7
kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)

oof = np.zeros((len(train_alt)))
pred = np.zeros((len(test_alt)))

FEATURES = CATS + ["Weight Capacity (kg)"] + COMBO

# OUTER K FOLD
# split train so that we can fit the endoding on one part and then transform the rest
# moving through all folds, eventuall all items in train will get transformed
for i, (fit_index, val_index) in enumerate(kf.split(train_alt)):
    print(f"### OUTER Fold {i+1} ###")

    # this will remove the ID as Index and also drop it from columns
    X_train = train_alt.loc[fit_index, FEATURES + ['Price']].reset_index(drop=True).copy()
    y_train = train_alt.loc[fit_index,'Price']

    X_valid = train_alt.loc[val_index, FEATURES].reset_index(drop=True).copy()
    y_valid = train_alt.loc[val_index,'Price']

    # this will remove the ID as Index and also drop it from columns
    X_test = test_alt[FEATURES].reset_index(drop=True).copy()

    # INNER K FOLD (TO PREVENT LEAKAGE WHEN USING PRICE in Target Encoding)
    kf2 = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
    for j, (fit_index2, val_index2) in enumerate(kf2.split(X_train)):
        print(f" ## INNER Fold {j+1} (outer fold {i+1}) ##")

        X_train2 = X_train.loc[fit_index2,  FEATURES + ['Price']].copy()
        X_valid2 = X_train.loc[val_index2,  FEATURES].copy()
        ### FEATURE SET 1 (uses weight capacity transformed by price) ###
        col = "Weight Capacity (kg)"
        # fit on one fold
        tmp = X_train2.groupby(col).Price.agg(STATS)
        tmp.columns = [f"TE1_wc_{s}" for s in STATS]
        # transform the other fold
        # eventuallY all merges will have values for col!! For now we use how="left"!!
        X_valid2 = X_valid2.merge(tmp, on=col, how="left")
        for c in tmp.columns:
            X_train.loc[val_index2, c] = X_valid2[c].values

        ### FEATURE SET 2 (uses the rest of the columns transformed by price) ###
        for col in COMBO:
            # fit on one fold
            tmp = X_train2.groupby(col).Price.agg(STATS2)
            tmp.columns = [f"TE2_{col}_{s}" for s in STATS2]
            # transform on the other fold
            # eventuallY all merges will have values for col!! For now we use how="left"!!
            X_valid2 = X_valid2.merge(tmp, on=col, how="left")
            for c in tmp.columns:
                X_train.loc[val_index2,c] = X_valid2[c].values

    ############# at this point all of X_train has been trasnormed by the repeated folds
    ############# X_val and X_test have not been transformed yet!!!!
    ######### we will now fit on train to transform the valid and test

    ### FEATURE SET 1 (uses price) ###
    col = "Weight Capacity (kg)"
    tmp = X_train.groupby(col).Price.agg(STATS)
    tmp.columns = [f"TE1_wc_{s}" for s in STATS]
    # fit on train but transform the valid on the same weight capacity value.
    # this will create missing values in valid and test, because we are merging on col value
    X_valid = X_valid.merge(tmp, on=col, how="left")
    X_test = X_test.merge(tmp, on=col, how="left")

    ### FEATURE SET 2 (uses price) ###
    for col in COMBO:
        tmp = X_train.groupby(col).Price.agg(STATS2)
        tmp.columns = [f"TE2_{col}_{s}" for s in STATS2]
        # fit on train but transform the valid on the same weight capacity value.
        # this will create missing values in valid and test, because we are merging on col value
        X_valid = X_valid.merge(tmp, on=col, how="left")
        X_test = X_test.merge(tmp, on=col, how="left")

    ### FEATURE SET 3 (does not use price) ###
    ### THESE FEAUTRES DO NOT REQUIRE A SEPARATE VALIDATION SET (they do not use the target - price)
    for col in CATS:
        col2 = "Weight Capacity (kg)"
        tmp = X_train.groupby(col)[col2].agg(STATS2)
        tmp.columns = [f"FE3_{col}_wc_{s}" for s in STATS2]
        X_train = X_train.merge(tmp, on=col, how="left")
        X_valid = X_valid.merge(tmp, on=col, how="left")
        X_test = X_test.merge(tmp, on=col, how="left")

    # CONVERT TO CATS SO XGBOOST RECOGNIZES THEM
    ## THEY ARE ALREADY FACTORIZED!!!!! - WE ARE ONLY CHANGING THE DTYPE!!!!!!!!!!!!!!!!
    X_train[CATS] = X_train[CATS].astype("category")
    X_valid[CATS] = X_valid[CATS].astype("category")
    X_test[CATS] = X_test[CATS].astype("category")

    # DROP PRICE THAT WAS USED FOR TARGET ENCODING
    X_train = X_train.drop(['Price'],axis=1)

    ### FIT MODEL WITH **EXISTING FOLD** FOR TRAIN AND VALIDATION !!!!!!!!!!!!!!!!!!!

    # BUILD MODEL
    model = XGBRegressor(
        device="cuda",
        max_depth=6,
        colsample_bytree=0.5,
        subsample=0.8,
        n_estimators=10_000,
        learning_rate=0.02,
        enable_categorical=True,
        min_child_weight=10,
        early_stopping_rounds=100,
    )

    # TRAIN MODEL
    COLS = X_train.columns
    model.fit(
        X_train[COLS], y_train,
        eval_set=[(X_valid[COLS], y_valid)],
        verbose=300,
    )

    # PREDICT OOF AND TEST
    # evaluation for out of fold
    oof[val_index] = model.predict(X_valid[COLS])
    # prediction
    pred += model.predict(X_test[COLS])

# final prediction score is the average of the in fold predictions
preds /= FOLDS

# OVERALL CROSS VALIDATION SCORE

In [ ]:
# COMPUTE OVERALL CV SCORE
true = train_alt.Price.values
s = np.sqrt(np.mean( (oof-true)**2.0 ) )
print(f"=> Overall CV Score = {s}")

## Feature Importance

In [ ]:
import xgboost as xgb
fig, ax = plt.subplots(figsize=(10, 20))
xgb.plot_importance(model, max_num_features=100, importance_type='gain',ax=ax)
plt.title("Top 100 Feature Importances (XGBoost)")
plt.show()

## make submission

In [ ]:
sub = pd.read_csv("/kaggle/input/playground-series-s5e2/sample_submission.csv")
sub.Price = pred
sub.to_csv(f"submission_v{VER}.csv",index=False)
sub.head()

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(sub.Price,bins=100)
plt.title("Test Predictions")
plt.show()